<font size = 5>Segmenting and Clustering Neighborhoods in Toronto (3)</font>

In [14]:
import pandas as pd

df=pd.read_csv("uber-raw-data-apr14.csv")

df=df.append(pd.read_csv("uber-raw-data-aug14.csv"), ignore_index=True)
df=df.append(pd.read_csv("uber-raw-data-jul14.csv"), ignore_index=True)
df=df.append(pd.read_csv("uber-raw-data-may14.csv"), ignore_index=True)
df=df.append(pd.read_csv("uber-raw-data-sep14.csv"), ignore_index=True)
df=df.append(pd.read_csv("uber-raw-data-jun14.csv"), ignore_index=True)


In [10]:
ldf.Base.unique()

array(['B02512', 'B02598', 'B02617', 'B02682', 'B02764'], dtype=object)

In [16]:
df["Date/Time"]= pd.to_datetime(df["Date/Time"])

In [21]:
df["year"]= df["Date/Time"].map(lambda x: x.year) 
df["month"]= df["Date/Time"].map(lambda x: x.month)
df["day"]= df["Date/Time"].map(lambda x: x.day)
df["hour"]= df["Date/Time"].map(lambda x: x.hour)
df["minute"]= df["Date/Time"].map(lambda x: x.minute)
df["second"]= df["Date/Time"].map(lambda x: x.second)

In [22]:
df.head()

,Date/Time,Lat,Lon,Base,year,month,day,hour,minute,second
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,2014,4,1,0,11,0
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,2014,4,1,0,17,0
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,2014,4,1,0,21,0
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,2014,4,1,0,28,0
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,2014,4,1,0,33,0


In [23]:
from sklearn.cluster import KMeans 
from sklearn.preprocessing import StandardScaler
import numpy as np

X = df.values[:,1:3]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset


array([[ 0.74441341,  0.31640287],
       [-0.3144126 , -1.07358489],
       [-0.191759  , -0.24937104],
       ...,
       [ 0.61925667, -0.27381806],
       [-0.32692828, -0.37335236],
       [ 0.02851686, -0.20571565]])

In [24]:
num_clusters = 5

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

In [25]:
df["Cluster Labels"]=labels

In [27]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\4713\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    conda-4.8.2                |           py37_0         3.0 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           T

In [31]:
# create map
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[40.7690, -73.9549], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Lat'], df['Lon'], df['Base'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyboardInterrupt: 